## Symbolic Aggregate Approximation

### 1.  [reference](http://dl.acm.org/citation.cfm?id=1285965)
### 2. main usage for time series data:
1. indexing and query
2. calculating distance between time-sereis and thus perform clustering/classification
3. symbolic representation for time series - inspiring text-mining related tasks such as association mining
4. vector representation of time-series
    
### 3. algorithm steps

1. Segment time-series data into gapless pieces (e.g., gap introduced by missing values or change of sampling frequences)

2. Each piece will be SAXed into a sequence of "words" (e.g., "abcdd" "aabcd", ...). This is done by rolling a sliding window of length $window$ with a stride of length $stride$. If $stride$ < $window$, there will be overlapping of different windows. Later each window will be converted into one word

3. for each sliding window:

    3.1 whiten/normalize across the window (it is the step key to many problems)
    
    3.2 discretize on time axis (index) by grouping points into equal-sized bins (bin sizes could be fractional) - controlled by $nbins$. For each bin, use the mean of bin as local approximation.
    
    3.3 discretize on value axis by dividing values into $nlevels$ quantiles (equiprobability), for each level, calculate the "letter" by $cutpoint$ table
    
    3.4 at the end, each bin in a sliding window will be mapped to a letter, each window in the piece of time-series will be mapped to a word, and the whole piece of series will be a sentence
    
    3.5 calcualte the distance between two symoblic representations by their corresponding levels
    
    3.6 if a vector representation is necessary, each letter can be mapped to a scalar value, such as the mean of the  corresponding level.

## sax module test

In [1]:
import matplotlib.pyplot as plt
%matplotlib inline

In [5]:
import org_pysax as pysax
import numpy as np
#reload(pysax)


In [6]:
sax = pysax.SAXModel(window=5, stride=2) 
sax.sym2vec

{'A': -0.67, 'B': -0.335, 'C': 0.335, 'D': 0.67}

In [7]:
## test normalization
sax = pysax.SAXModel(window=3, stride=2) 
list(sax.sliding_window_index(10))
ws = np.array([-1.0, 0.9 , 0.5, 0.5, 0.3, 0.2, 0.1, 1.0, 0.8, 0.7])
print(ws.mean(), ws.std())
ss = sax.whiten(ws)
print(ss.mean(), ss.std()) 
print("\n random variable: \n{0} \n normalized variable: \n{1}". format(ws,ss))

0.4 0.545893762558
-2.22044604925e-17 0.999999999817

 random variable: 
[-1.   0.9  0.5  0.5  0.3  0.2  0.1  1.   0.8  0.7] 
 normalized variable: 
[-2.56460157  0.91592913  0.18318583  0.18318583 -0.18318583 -0.36637165
 -0.54955748  1.09911496  0.73274331  0.54955748]


In [8]:
## explore binning
from fractions import Fraction
def binpack(xs, nbins):
    xs = np.asarray(xs)
    binsize = Fraction(len(xs), nbins)
    wts = [1 for _ in range(int(binsize))] + [binsize-int(binsize)]
    pos = 0
    while pos < len(xs):
        if wts[-1] == 0:
            n = len(wts) - 1
        else:
            n = len(wts)
        yield zip(xs[pos:(pos+n)], wts[:n])
        pos += len(wts) - 1
        rest_wts = binsize-(1-wts[-1])
        wts = [1-wts[-1]] + [1 for _ in range(int(rest_wts))] + [rest_wts-int(rest_wts)]


In [9]:
xs = list(range(0, 16))
print("\nRange:{0} \nbinpack (5): ". format(xs))
r = list(list(binpack(xs, 5)))
for item in r:
    print(list(item))

print("\nbinpack (4)")
xs = range(0, 16)
r = list(binpack(xs, 4))
for item in r:
    print(list(item))

print("\nbinpack (3)")
xs = range(0, 5)
r = list(binpack(xs, 3))
for item in r:
    print(list(item))


Range:[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15] 
binpack (5): 
[(0, 1), (1, 1), (2, 1), (3, Fraction(1, 5))]
[(3, Fraction(4, 5)), (4, 1), (5, 1), (6, Fraction(2, 5))]
[(6, Fraction(3, 5)), (7, 1), (8, 1), (9, Fraction(3, 5))]
[(9, Fraction(2, 5)), (10, 1), (11, 1), (12, Fraction(4, 5))]
[(12, Fraction(1, 5)), (13, 1), (14, 1), (15, 1)]

binpack (4)
[(0, 1), (1, 1), (2, 1), (3, 1)]
[(4, Fraction(1, 1)), (5, 1), (6, 1), (7, 1)]
[(8, Fraction(1, 1)), (9, 1), (10, 1), (11, 1)]
[(12, Fraction(1, 1)), (13, 1), (14, 1), (15, 1)]

binpack (3)
[(0, 1), (1, Fraction(2, 3))]
[(1, Fraction(1, 3)), (2, 1), (3, Fraction(1, 3))]
[(3, Fraction(2, 3)), (4, 1)]


In [ ]:
## test binning
sax = pysax.SAXModel(nbins=3) 
print(list(sax.binpack(np.ones(5))))
print('\n') 
print(list(sax.binpack(np.ones(9))))

In [ ]:
## explore symbolization
import pandas as pd
cutpoints = [-np.inf, -0.43, 0.43, np.inf]
#xs = np.random.random(10)
xs = ws
v = pd.cut(xs, bins = cutpoints, labels=["A", "B", "C"])
print("{0}\n{1}".format(xs,v))

In [ ]:
#xs = np.random.randn(10)
print(xs)
sax = pysax.SAXModel(window=2, stride=2) 
sax.symbolize(xs)

In [ ]:
sax = pysax.SAXModel(nbins = 5, alphabet="abcd")
#xs = np.random.randn(20) * 2 + 1.
print(xs)
sax.symbolize_window(xs)

In [ ]:
sax = pysax.SAXModel(window=3, stride = 3, nbins = 2, alphabet="ABCD")
#xs = np.random.randn(103) * 2 + np.arange(103) * 0.03
print(xs)
plt.plot(xs)
print(sax.symbolize_signal(xs))
print(sax.symbolize(xs))

In [ ]:
#reload(pysax)
#sax = pysax.SAXModel(window=20, stride = 5, nbins = 5, alphabet="ABCD")
#xs = np.random.randn(103) * 2 + np.arange(103) * 0.03
words = sax.symbolize_signal(xs)
ts_indices = sax.convert_index(word_indices=range(len(words)))
word_indices = sax.convert_index(ts_indices = range(len(xs)))
print(words)
plt.plot(word_indices)
print("size of xs {0} \n ts_indices: {1}".format(len(xs),ts_indices))
print("\nword_indices:\n",word_indices)

#The parallel functionality only works in unix/linux systems the following part is only to see the performance of this approach

In [ ]:

import pysax
import numpy as np 
from joblib import Parallel, delayed
#reload(pysax)
sax = pysax.SAXModel(window=20, stride = 5, nbins = 5, alphabet="ABCD")
N = 10000
xs = np.random.randn(N) * 2 + np.arange(N) * 0.03
#plt.plot(xs)
%time psymbols = sax.symbolize_signal(xs, parallel="joblib", n_jobs=30)
#%time psymbols = sax.symbolize_signal(xs, n_jobs=30)


In [ ]:
sax = pysax.SAXModel(window=20, stride = 5, nbins = 5, alphabet="ABCD")
#xs = np.random.randn(1000000) * 2 + np.arange(1000000) * 0.03
#plt.plot(xs)
%time symbols = sax.symbolize_signal(xs)
print(np.all(psymbols==symbols))

In [ ]:
## test symbol to vector
%time vecs = sax.symbol_to_vector(psymbols)
vecs.shape

In [ ]:
## test symbol distance
#reload(pysax)
sax = pysax.SAXModel(window=20, stride = 5, nbins = 5, alphabet="ABCD")
sax.symbol_distance(psymbols[0], psymbols[1]), sax.symbol_distance(psymbols[1], psymbols[2])

In [ ]:
v1, v2, v3 = sax.symbol_to_vector(psymbols[:3])

In [ ]:
np.sqrt(np.sum( (v1-v2)**2 )), np.sqrt(np.sum( (v2-v3)**2 ))

In [ ]:
psymbols[:3]

In [ ]:
## test paa vectors
import pysax
import numpy as np 
#reload(pysax)
sax = pysax.SAXModel(window=20, stride = 5, nbins = 5, alphabet="ABCD")
xs = np.random.randn(N) * 2 + np.arange(N) * 0.03
#plt.plot(xs)
#%time vecs = sax.signal_to_paa_vector(xs, n_jobs=30)
time vecs = sax.signal_to_paa_vector(xs, n_jobs=30)

In [ ]:
vecs[:10, :]

In [ ]:
psymbols[:10]